<a href="https://colab.research.google.com/github/Brandon-Bernal-Alarcon/Notas/blob/main/Machine%20Learning/Evaluacion_de_Modelos/Introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Evaluación y selección de modelos**

En el aprendizaje automático supervisado, la **evaluación del modelo** constituye una etapa central dentro del flujo de trabajo, una vez entrenado un modelo, es indispensable medir su desempeño para determinar si satisface los objetivos reales de la aplicación, esta evaluación no solo permite comparar distintos modelos entrenados, sino también seleccionar configuraciones de hiperparámetros y orientar mejoras posteriores en la fase de refinamiento del modelo.

Aunque métricas simples como la precisión (accuracy) en clasificación o el coeficiente de determinación $R^2$ en regresión son comúnmente utilizadas, estas medidas pueden ser insuficientes para describir adecuadamente el comportamiento de un modelo en escenarios reales.

## **Limitaciones de las métricas simples**

La precisión se define como la fracción de instancias correctamente clasificadas sobre el total de observaciones N.

$$Acurrancy = \frac{\text{Predicciones correctas}}{\text{Total de Predicciones}}$$

Si bien es fácil de interpretar, esta métrica presenta limitaciones importantes:

- No distingue entre distintos tipos de errores.

- Asume implícitamente que todos los errores tienen el mismo costo.

- Puede ser altamente engañosa en conjuntos de datos con clases desbalanceadas.

En aplicaciones reales, como sistemas de búsqueda, recomendación, detección de fraude o diagnóstico médico, los costos asociados a errores de distinta naturaleza suelen ser muy diferentes, por tanto, una métrica única como la precisión no siempre refleja el verdadero desempeño del modelo.

**Ejemplo práctico: conjunto de datos de dígitos**

Se utilizará el conjunto de datos de dígitos de sklearn, originalmente, este conjunto tiene diez clases balanceadas (dígitos del 0 al 9), sin embargo, para ilustrar el problema del desbalance, se transforma en un problema binario:

Clase positiva: dígito 1

Clase negativa: todos los demás dígitos

In [2]:
%matplotlib notebook
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

dataset = load_digits()
X, y = dataset.data, dataset.target

for class_name, class_count in zip(dataset.target_names, np.bincount(dataset.target)):
    print(class_name,class_count)

0 178
1 182
2 177
3 183
4 181
5 182
6 181
7 179
8 174
9 180


In [3]:
y_binary_imbalanced = y.copy()
y_binary_imbalanced[y_binary_imbalanced != 1] = 0

print('Original labels:\t', y[1:30])
print('New binary labels:\t', y_binary_imbalanced[1:30])

Original labels:	 [1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]
New binary labels:	 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [4]:
np.bincount(y_binary_imbalanced)

array([1615,  182])

Este conteo muestra un fuerte desbalance: muchas instancias negativas y relativamente pocas positivas.

**Entrenamiento de un clasificador real**

Se entrena un clasificador SVM con kernel RBF sobre este conjunto binario desbalanceado y se evalúa su precisión:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)

# Accuracy of Support Vector Machine classifier
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1).fit(X_train, y_train)
svm.score(X_test, y_test)

0.9955555555555555

El valor obtenido es cercano al 99%, lo cual podría parecer un buen desempeño a primera vista.

**Clasificador ficticio como línea base (DummyClassifier)**

Para evaluar si este resultado es realmente significativo, se introduce el uso de DummyClassifier, que proporciona una línea base de desempeño nulo.

DummyClassifier es un clasificador que realiza predicciones utilizando reglas simples, que pueden ser útiles como base para la comparación con clasificadores reales, especialmente con clases desequilibradas.

In [6]:
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test)

y_dummy_predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
dummy_majority.score(X_test, y_test)

0.9044444444444445

Este clasificador siempre predice la clase mayoritaria (clase negativa). Sorprendentemente, su precisión también es cercana al 90%, lo que demuestra que la precisión del SVM no es particularmente impresionante en este contexto.

**Otras estrategias de DummyClassifier**

Otras estrategias útiles para análisis comparativo:



```
DummyClassifier(strategy='stratified')
DummyClassifier(strategy='uniform')
DummyClassifier(strategy='constant', constant=1)
```

Estas estrategias permiten:

- Simular predicciones aleatorias según la distribución de clases.

- Forzar predicciones positivas para poder calcular métricas como F-score.

- Obtener referencias útiles para entender los tipos de errores esperados.

**Comparación con un modelo mejor ajustado**

Al modificar el kernel del SVM a uno lineal, el desempeño mejora significativamente:

In [8]:
svm = SVC(kernel='linear', C=1).fit(X_train, y_train)
svm.score(X_test, y_test)

0.9777777777777777

En este caso, la precisión se eleva hasta aproximadamente 98%, superando claramente la línea base del DummyClassifier, lo que indica que el modelo sí está capturando patrones relevantes.

Este análisis muestra que:

- Una precisión alta no garantiza un buen modelo.

- Siempre debe compararse el desempeño contra una línea base trivial.

- Un modelo cercano al rendimiento de un DummyClassifier suele indicar:<br>
       - características poco informativas,<br>
       - mala elección de kernel,<br>
       - hiperparámetros inadecuados, <br>
       - o fuerte desbalance de clases.

Por ello, en problemas desbalanceados es fundamental utilizar métricas adicionales más allá de la precisión.

## **Resultados clave**

La evaluación es una etapa crítica del flujo de trabajo.

Métricas simples pueden ser insuficientes o engañosas.

El contexto de la aplicación determina qué métricas son relevantes.

Los clasificadores ficticios proporcionan una referencia esencial.

Es necesario un análisis más detallado de los errores del modelo.

## **Métricas de evaluación vs funciones objetivo**

Un aspecto clave es la diferencia entre **Función objetivo de entrenamiento** que es aquella que el algoritmo optimiza directamente durante el ajuste del modelo (por ejemplo, una función de pérdida convexa) y **Métrica de evaluación** que es aquella que se utiliza para juzgar el éxito del modelo en función de los objetivos de la aplicación.

En muchos casos, **la función objetivo** se elige por razones computacionales o de optimización, mientras que la **métrica de evaluación** está alineada con criterios de negocio, impacto clínico o experiencia del usuario, por ello, ambas no necesariamente coinciden.

Un ejemplo claro es el de los motores de búsqueda comerciales, el modelo puede entrenarse para predecir relevancia de documentos, pero su evaluación final puede involucrar métricas indirectas como duración de sesiones, tasas de clics o satisfacción del usuario.

## **Importancia del contexto de la aplicación**

La elección de métricas de evaluación debe estar guiada por los objetivos específicos del problema:

En tareas donde todas las clases son igualmente importantes y balanceadas (por ejemplo, reconocimiento de dígitos manuscritos), la precisión puede ser suficiente.

En aplicaciones médicas, puede ser prioritario minimizar falsos negativos, incluso a costa de aumentar falsos positivos.

En aplicaciones orientadas al usuario (recomendaciones, publicidad, búsqueda), los falsos positivos pueden ser especialmente perjudiciales para la experiencia del usuario.

Por esta razón, en la práctica se suelen utilizar paneles de métricas múltiples en lugar de una sola medida de desempeño.

## **Problemas con clases desbalanceadas**

El concepto de clases desbalanceadas es muy común en problemas reales de Machine Learning. En estos escenarios, la clase negativa suele ser ampliamente mayoritaria, mientras que la clase positiva es rara.

Ejemplos típicos incluyen:

- Detección de fraude con tarjetas de crédito.

- Recomendaciones de productos relevantes.

- Identificación de eventos poco frecuentes.

En estos casos, un clasificador que siempre predice la clase mayoritaria puede alcanzar una precisión muy alta sin aportar ningún valor predictivo real.

## **Clasificadores ficticios como línea base**

Para evidenciar este problema, se introduce el uso de **DummyClassifier** en scikit-learn, estos clasificadores no aprenden patrones a partir de los datos, sino que siguen reglas simples como:

- Predecir siempre la clase más frecuente.

- Predecir clases de manera aleatoria según su distribución.

- Predecir clases de forma uniforme.

El desempeño de un DummyClassifier define una línea base de precisión nula, que sirve como comprobación de cordura, si un modelo entrenado obtiene un rendimiento cercano al de un clasificador ficticio, esto indica que el modelo no está aprendiendo información relevante o que existen problemas con las características, el kernel o los hiperparámetros.

## **Implicaciones para la selección de modelos**

El análisis presentado conduce a que la selección de modelos no debe basarse únicamente en la precisión, especialmente en problemas desbalanceados, es necesario utilizar métricas adicionales que permitan entender los tipos de errores que comete el modelo, comparar modelos en función de los costos reales de esos errores, elegir configuraciones que optimicen el desempeño relevante para la aplicación.

Este razonamiento motiva el estudio detallado de la matriz de confusión y de métricas como precisión, recall y F-score, que se desarrollan en el siguiente video.